##数据预处理

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from data_features import featureRMS, featureMAV, featureWL, featureZC, featureSSC

def deploy_normalize(X, min_path, max_path):
    """
    在部署端使用保存的最大值和最小值进行归一化。
    """
    data_min = np.load(min_path)
    data_max = np.load(max_path)

    denominator = data_max - data_min
    denominator[denominator == 0] = 1
    X_scaled = (X - data_min) / denominator
    return X_scaled

# 文件路径
MODEL_PATH = "my_model.tflite"
RAW_DATA_PATH = "pre/test/up.csv"
min_path ="train/scaler_min_20250708_182347.npy"
max_path ="train/scaler_max_20250708_182347.npy"

# 窗口设置
WINDOW_SIZE = 5
STRIDE = 2

# 1. 加载原始 CSV 数据（应为 36列）
df = pd.read_csv(RAW_DATA_PATH, header=None)
assert df.shape[1] == 36, f"期望36列，实际是{df.shape[1]}列"
df = df.apply(pd.to_numeric, errors='coerce')
data = df.values  # shape: (n_samples, 36)

# 2. 加载 TFLite 模型
interpreter = tf.lite.Interpreter(model_path=MODEL_PATH)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(f"Model input shape: {input_details[0]['shape']}")  # 调试模型输入

# 3. 滑动窗口预测
predictions = []

for i in range(0, len(data) - WINDOW_SIZE + 1, STRIDE):
    window = data[i:i + WINDOW_SIZE]  # shape: (WINDOW_SIZE, 36)
    feature_list = []

    try:
        rms = featureRMS(window)
        mav = featureMAV(window)
        wl = featureWL(window)
        zc = featureZC(window)
        ssc = featureSSC(window)
        features_180d = np.hstack((rms, mav, wl, zc, ssc))
        if features_180d.shape[0] != 180:  # 5 特征 * 36 通道
            print(f"警告： 窗口 {i} 特征维度  不正确")
            continue
        feature_list.append(features_180d)
    except Exception as e:
        print(f" 窗口 {i} 特征提取失败: {e}")

    feature_list = np.array(feature_list)
    feature_list = deploy_normalize(feature_list,min_path, max_path)
    input_tensor = feature_list.reshape(1, 2, 90).astype(np.float32)  # shape: (1, 2, 90)
    print(f"Input tensor shape: {input_tensor.shape}")

    # 模型推理
    interpreter.set_tensor(input_details[0]['index'], input_tensor)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])  # shape: [1, 3]

    pred_class = np.argmax(output)
    confidence = np.max(output)
    predictions.append((i, pred_class, confidence))

# 4. 输出预测结果
print("滑动窗口预测结果：")
for i, cls, conf in predictions:
    print(f"窗口{i}: 预测类别 → {cls}, 置信度: {conf:.2f}")

# 5. 统计特定类别的概率
class_counts = {}
total_predictions = len(predictions)

for _, cls, _ in predictions:
    if cls in class_counts:
        class_counts[cls] += 1
    else:
        class_counts[cls] = 1

# 计算每个类别的概率
class_probabilities = {cls: count / total_predictions for cls, count in class_counts.items()}

# 输出每个类别的概率
print("\n特定类别的概率：")
for cls, prob in class_probabilities.items():
    print(f"类别 {cls}: 概率 → {prob:.2%}")


C:\Users\-\.conda\envs\python\Lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


Model input shape: [ 1  2 90]
Input tensor shape: (1, 2, 90)
Input tensor shape: (1, 2, 90)
Input tensor shape: (1, 2, 90)
Input tensor shape: (1, 2, 90)
Input tensor shape: (1, 2, 90)
Input tensor shape: (1, 2, 90)
Input tensor shape: (1, 2, 90)
Input tensor shape: (1, 2, 90)
Input tensor shape: (1, 2, 90)
Input tensor shape: (1, 2, 90)
Input tensor shape: (1, 2, 90)
Input tensor shape: (1, 2, 90)
Input tensor shape: (1, 2, 90)
Input tensor shape: (1, 2, 90)
Input tensor shape: (1, 2, 90)
Input tensor shape: (1, 2, 90)
Input tensor shape: (1, 2, 90)
Input tensor shape: (1, 2, 90)
Input tensor shape: (1, 2, 90)
Input tensor shape: (1, 2, 90)
Input tensor shape: (1, 2, 90)
Input tensor shape: (1, 2, 90)
Input tensor shape: (1, 2, 90)
Input tensor shape: (1, 2, 90)
Input tensor shape: (1, 2, 90)
Input tensor shape: (1, 2, 90)
Input tensor shape: (1, 2, 90)
Input tensor shape: (1, 2, 90)
Input tensor shape: (1, 2, 90)
Input tensor shape: (1, 2, 90)
Input tensor shape: (1, 2, 90)
Input ten

##  H5

In [58]:
import numpy as np
import pandas as pd
import tensorflow as tf
from data_features import featureRMS, featureMAV, featureWL, featureZC, featureSSC

def deploy_normalize(X, min_path, max_path):
    """
    在部署端使用保存的最大值和最小值进行归一化。
    """
    data_min = np.load(min_path)
    data_max = np.load(max_path)

    denominator = data_max - data_min
    denominator[denominator == 0] = 1
    X_scaled = (X - data_min) / denominator
    return X_scaled

# 文件路径
MODEL_PATH = "train/20250730_230611/lstm_model_categorical_crossentropy.h5"
RAW_DATA_PATH = "pre/test/dun.csv"
min_path = "train/scaler_min_20250708_182347.npy"
max_path = "train/scaler_max_20250708_182347.npy"

# 窗口设置
WINDOW_SIZE = 5
STRIDE = 2

# 1. 加载原始 CSV 数据（应为 36列）
df = pd.read_csv(RAW_DATA_PATH, header=None)
assert df.shape[1] == 36, f"期望36列，实际是{df.shape[1]}列"
df = df.apply(pd.to_numeric, errors='coerce')
data = df.values  # shape: (n_samples, 36)

# 2. 加载 HDF5 模型
model = tf.keras.models.load_model(MODEL_PATH)
input_shape = model.input_shape
print(f"模型输入形状: {input_shape}")  # 调试模型输入

# 3. 滑动窗口预测
predictions = []

for i in range(0, len(data) - WINDOW_SIZE + 1, STRIDE):
    window = data[i:i + WINDOW_SIZE]  # shape: (WINDOW_SIZE, 36)
    feature_list = []

    try:
        rms = featureRMS(window)
        mav = featureMAV(window)
        wl = featureWL(window)
        zc = featureZC(window)
        ssc = featureSSC(window)
        features_180d = np.hstack((rms, mav, wl, zc, ssc))
        if features_180d.shape[0] != 180:  # 5 特征 * 36 通道
            print(f"警告：窗口 {i} 特征维度不正确")
            continue
        feature_list.append(features_180d)
    except Exception as e:
        print(f"窗口 {i} 特征提取失败: {e}")
        continue

    feature_list = np.array(feature_list)
    feature_list = deploy_normalize(feature_list, min_path, max_path)
    input_tensor = feature_list.reshape(1, 2, 90).astype(np.float32)  # shape: (1, 2, 90)
    print(f"输入张量形状: {input_tensor.shape}")

    # 模型推理
    output = model.predict(input_tensor, verbose=0)  # shape: [1, 3]
    pred_class = np.argmax(output, axis=1)[0]
    confidence = np.max(output)
    predictions.append((i, pred_class, confidence))

# 4. 输出预测结果
print("滑动窗口预测结果：")
for i, cls, conf in predictions:
    print(f"窗口{i}: 预测类别 → {cls}, 置信度: {conf:.2f}")

# 5. 统计特定类别的概率
class_counts = {}
total_predictions = len(predictions)

for _, cls, _ in predictions:
    if cls in class_counts:
        class_counts[cls] += 1
    else:
        class_counts[cls] = 1

# 计算每个类别的概率
class_probabilities = {cls: count / total_predictions for cls, count in class_counts.items()}

# 输出每个类别的概率
print("\n特定类别的概率：")
for cls, prob in class_probabilities.items():
    print(f"类别 {cls}: 概率 → {prob:.2%}")

模型输入形状: (None, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (1, 2, 90)
输入张量形状: (

##预测

input_data  shape: (152, 25, 9)
预测结果形状: (152, 1, 2)
预测结果: [[[2.0715040e-04 9.9979287e-01]]

 [[1.9913088e-04 9.9980086e-01]]

 [[1.9889844e-04 9.9980110e-01]]

 [[2.0117672e-04 9.9979883e-01]]

 [[2.0379854e-04 9.9979621e-01]]

 [[2.0898826e-04 9.9979109e-01]]

 [[2.2319258e-04 9.9977678e-01]]

 [[2.0839271e-04 9.9979168e-01]]

 [[2.2274337e-04 9.9977726e-01]]

 [[2.2981589e-04 9.9977022e-01]]

 [[2.1159765e-04 9.9978846e-01]]

 [[2.1475929e-04 9.9978524e-01]]

 [[2.0825268e-04 9.9979180e-01]]

 [[2.1263091e-04 9.9978739e-01]]

 [[2.0922533e-04 9.9979085e-01]]

 [[2.1267775e-04 9.9978739e-01]]

 [[2.1812847e-04 9.9978191e-01]]

 [[2.1221732e-04 9.9978775e-01]]

 [[2.1971129e-04 9.9978036e-01]]

 [[2.3441705e-04 9.9976557e-01]]

 [[2.3411127e-04 9.9976593e-01]]

 [[2.2586923e-04 9.9977416e-01]]

 [[2.2834114e-04 9.9977165e-01]]

 [[2.2266843e-04 9.9977738e-01]]

 [[2.0314244e-04 9.9979693e-01]]

 [[1.9598167e-04 9.9980408e-01]]

 [[1.9899746e-04 9.9980098e-01]]

 [[2.0303958e-04 9.99796

C:\Users\-\.conda\envs\python\Lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [47]:
import tensorflow as tf

interpreter = tf.lite.Interpreter(model_path="my_model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("输入信息：")
print("  shape:", input_details[0]['shape'])
print("  dtype:", input_details[0]['dtype'])

print("\n输出信息：")
print("  shape:", output_details[0]['shape'])
print("  dtype:", output_details[0]['dtype'])


输入信息：
  shape: [ 1  2 90]
  dtype: <class 'numpy.float32'>

输出信息：
  shape: [1 3]
  dtype: <class 'numpy.float32'>


C:\Users\-\.conda\envs\python\Lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
